<a href="https://colab.research.google.com/github/airballoh87/bexa/blob/main/painted_tree_bexa_boutique_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [ ]:
##data and stats
import pandas as pd
import seaborn as sns

##visualis
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # Import the ticker module

##data sources
import gspread
from google.colab import drive

##web scraping
import requests
from bs4 import BeautifulSoup

##machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

##NLP
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk

##saving models
import joblib

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Get data

##For training

In [5]:
gc  = gspread.authorize(creds)

# Function to read data from a Google Sheet and add a store location column
def read_sheet(url, worksheet_name, location):
    gsheets = gc.open_by_url(url)
    sheets = gsheets.worksheet(worksheet_name).get_all_values()
    df = pd.DataFrame(sheets[1:], columns=sheets[0])
    df['store_location'] = location
    return df

# URLs and worksheet names
kentucky_url = 'https://docs.google.com/spreadsheets/d/117PfZR2st0KhQ1L4t3GuNL7vsj2CYq6MDJhMm-cr6N4/edit?gid=427915297#gid=427915297'
cincinnati_url = 'https://docs.google.com/spreadsheets/d/1r_xXGRpcoegezaQTwpoOXhgaWpbHZIJYDoVnygquSGQ'
kentucky_sheet = 'Consignor Access - Activity (10)'
cincinnati_sheet = 'Consignor Access - Activity (11)'

# Read data from both sheets
kentucky_data = read_sheet(kentucky_url, kentucky_sheet, 'Kentucky')
cincinnati_data = read_sheet(cincinnati_url, cincinnati_sheet, 'Cincinnati')

# Combine the data into one DataFrame
combined_historical_data = pd.concat([kentucky_data, cincinnati_data], ignore_index=True)

# Filter out rows where 'Retail Price' is NA, NULL, or an empty string
combined_historical_data = combined_historical_data[combined_historical_data['Retail Price'].notna() & (combined_historical_data['Retail Price'] != '')]

# Remove rows where 'Category' is NA, NULL, or an empty string
combined_historical_data = combined_historical_data.dropna(subset=['Category'])
combined_historical_data = combined_historical_data[combined_historical_data['Category'] != '']

# Filter out rows where 'Retail Price' has a value of 'Retail Price'
combined_historical_data = combined_historical_data[combined_historical_data['Retail Price'] != 'Retail Price']

# Display the first few rows and get the column names and data types
print(combined_historical_data.info())
print(combined_historical_data.head())


<class 'pandas.core.frame.DataFrame'>
Index: 3994 entries, 0 to 4300
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              3994 non-null   object
 1   Description       3994 non-null   object
 2   SKU               3994 non-null   object
 3   Invoice           3994 non-null   object
 4   Category          3994 non-null   object
 5   Type              3994 non-null   object
 6   Theme             3994 non-null   object
 7   Retail Price      3994 non-null   object
 8   Sold Price        3994 non-null   object
 9   Store Amount      3994 non-null   object
 10  Consignor Amount  3994 non-null   object
 11  store_location    3994 non-null   object
dtypes: object(12)
memory usage: 405.6+ KB
None
        Date                                        Description     SKU  \
0   4/6/2024       100% leather mini zip short wallet:(2-01445)  9LN7ZA   
1  4/10/2024  11 - PREMIUM -Crystal Rivoli Stone 12MM Br

## Real Data

### Web Scrape from Painted Tree = Cincy only

In [3]:
# Your login credentials
USERNAME = '105357'
PASSWORD = 'WinstonWillow1!'

# Headers for HTTP requests
http_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}

# Initialize an empty DataFrame to accumulate the data
accumulated_data = pd.DataFrame()

# Start a session to keep the cookies
with requests.Session() as session:
    # Prepare login data
    login_data = {
        'consignorId': USERNAME,
        'password': PASSWORD
    }

    # Send a POST request to the login endpoint
    login_url = 'https://cincinnati.consignoraccess.com/auth'  # Adjust this URL if necessary
    login_response = session.post(login_url, headers=http_headers, data=login_data)

    # Verify login by checking the response and cookies
    print("Login response status code:", login_response.status_code)
    print("Cookies after login:", session.cookies.get_dict())

    # Proceed to the data URL
    #data_url = 'https://cincinnati.consignoraccess.com/activity'
    data_url = 'https://cincinnati.consignoraccess.com/activity?fromDate=01/01/2023&toDate=06/30/2024&showPaidItems=true'
    response = session.get(data_url, headers=http_headers)

    # Print the response status code and content for verification
    print("Data URL response status code:", response.status_code)
    print("Data URL response content snippet:", response.content[:500])  # Print first 500 characters of the response content

    soup = BeautifulSoup(response.content, 'html.parser')
    #print("Soup object:", soup.prettify()[:500])  # Print first 500 characters of the parsed HTML

    table = soup.find('table', {'id': 'make-responsive'})

    if table:
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = []
        for row in table.find('tbody').find_all('tr'):
            columns = row.find_all('td')
            if columns:
                rows.append([col.text.strip() for col in columns])
        accumulated_data = pd.DataFrame(rows, columns=headers)

      # Add new columns 'Category', 'Type', and 'Theme' with blank values
        accumulated_data['Category'] = ''
        accumulated_data['Type'] = ''
        accumulated_data['Theme'] = ''
        accumulated_data['Location'] = 'Cincinnati'

        print("Accumulated data:", accumulated_data)
    else:
        print("Table not found in the page.")

    # Optionally save to CSV
    # accumulated_data.to_csv("sales_activity_data.csv", index=False)


Login response status code: 200
Cookies after login: {'key': 'cuWNu2mDKxWI9TXm6wvt'}
Data URL response status code: 200
Data URL response content snippet: b'\n\n<!DOCTYPE html>\n<html lang="en">\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=EDGE"/>\n\t\t<title>Consignor Access - Activity</title>\n\n\t\t<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>\n\t\t<meta content="Consignor Access" name="description"/>\n\t\t<meta content="SimpleConsign" name="author"/>\n\n        \n\t\t\n\t\t\n\n\t\t<link rel="shortcut icon" href="/assets/favicon-9f8530d404b76a186ff295fe37f2b620.ico" type="image/x-icon">\n\n  \t\t<link re'
Accumulated data:            Date                                        Description     SKU  \
0     6/23/2024  Pink/Yellow/Champagne Sequin & Beaded Large To...  NV7VAG   
1     6/22/2024                                PIn - Dog Lover Pin  2TESGR   
2     6/22/2024    bracelets- red white and blue str

### Real data - Kentucky - from google sheet


In [7]:
gc  = gspread.authorize(creds)

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gzZ4ffnM5Uc4cCHmAigMQ4IS3xxVokBc1wM5tQjPQcM/edit?gid=85248025#gid=85248025') ##Kentucky
sheets = gsheets.worksheet('Painted Tree Category From Website - Kentucky').get_all_values()
kentucky_sales_data = pd.DataFrame(sheets[1:], columns=sheets[0])
kentucky_sales_data


,Date,Description,SKU,Invoice,Retail Price,Sold Price,Store Amount,Consignor Amount,Category,Type,Theme,Location
0,5/31/24,HEADBAND LUXURY STYLE - SALE CLEARANCE AS-IN -...,V67JCR,50020,$10.00,$10.00,-$1.00,$9.00,,,,Kentucky
1,5/31/24,kids headband - Ric Rac Rainbow Knotted Headband,E6SD2W,50020,$12.00,$12.00,-$1.20,$10.80,,,,Kentucky
2,5/31/24,kids - pink tulle headband with bow,VDKNDW,49917,$14.00,$14.00,-$1.40,$12.60,,,,Kentucky
3,5/31/24,kids - pink tulle headband with bow,VDKNDW,50026,$14.00,$14.00,-$1.40,$12.60,,,,Kentucky
4,5/31/24,bag - Swiftie Canvas bag,K1XNAV,49917,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky
...,...,...,...,...,...,...,...,...,...,...,...,...
2160,6/22/24,headband - purple gemstone embellished knotted,R9UD13,54938,$25.00,$25.00,-$2.50,$22.50,,,,Kentucky
2161,6/22/24,bride rhinestone sunglasses,GYX2AP,55194,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
2162,6/22/24,headband - American USA Flag Colored Knot Sequ...,R9LC27,55047,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
2163,6/22/24,earrings - orange cut out side tiger face earr...,L5H23K,55199,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky


###Merge data together

In [8]:
# Concatenate accumulated_data with historical_sales_data as union
union_data = pd.concat([accumulated_data, kentucky_sales_data], axis=0, ignore_index=True)
union_data

,Date,Description,SKU,Invoice,Retail Price,Sold Price,Store Amount,Consignor Amount,Category,Type,Theme,Location
0,6/23/2024,Pink/Yellow/Champagne Sequin & Beaded Large To...,NV7VAG,84936,$75.00,$75.00,-$7.50,$67.50,,,,Cincinnati
1,6/22/2024,PIn - Dog Lover Pin,2TESGR,84815,$6.50,$6.50,-$0.65,$5.85,,,,Cincinnati
2,6/22/2024,bracelets- red white and blue stretch bracelets,THYGCK,84863,$10.00,$10.00,-$1.00,$9.00,,,,Cincinnati
3,6/22/2024,Pink tiger acrylic earrings:(1-01561),1HGLFS,84792,$15.00,$15.00,-$1.50,$13.50,,,,Cincinnati
4,6/22/2024,earrings- small studs,1P8ABR,84903,$15.00,$15.00,-$1.50,$13.50,,,,Cincinnati
...,...,...,...,...,...,...,...,...,...,...,...,...
4816,6/22/24,headband - purple gemstone embellished knotted,R9UD13,54938,$25.00,$25.00,-$2.50,$22.50,,,,Kentucky
4817,6/22/24,bride rhinestone sunglasses,GYX2AP,55194,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
4818,6/22/24,headband - American USA Flag Colored Knot Sequ...,R9LC27,55047,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
4819,6/22/24,earrings - orange cut out side tiger face earr...,L5H23K,55199,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky


#Clean Data

## rows and columns - general

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import joblib


# Load the dataset
new_sales_data = combined_historical_data

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

new_sales_data['Retail Price'] = new_sales_data['Retail Price'].apply(clean_price)
new_sales_data['Sold Price'] = new_sales_data['Sold Price'].apply(clean_price)
new_sales_data['Store Amount'] = new_sales_data['Store Amount'].apply(clean_price)
new_sales_data['Consignor Amount'] = new_sales_data['Consignor Amount'].apply(clean_price)

# Drop rows with missing values in the target column 'Category'
new_sales_data = new_sales_data.dropna(subset=['Category'])

#Eliminates historical rows where the was a return
new_sales_data = new_sales_data[(new_sales_data['Sold Price'] >= 0) #&
                                #(new_sales_data['Store Amount'] >= 0) &
                                #(new_sales_data['Consignor Amount'] >= 0)
                                ]


# Clean up the 'Category' column
new_sales_data['Category'] = new_sales_data['Category'].str.upper()

# Make specified updates to the 'Category' column
category_updates = {
    'MISC': 'MISCELLANEOUS',
    'SHELL': 'DECOR',
    'CLOTHES': 'CLOTHING',
    'HAT': 'HATS',
    'APPAREL': 'CLOTHING',
    'DECOR - EASTER': 'DECOR',
    'DECOR - VALENTINES': 'DECOR',
    'DOGS': 'PET',
    'HAIR ACCESSORIES': 'ACCESSORIES: HAIR',
    'HOLIDAY - EASTER': 'DECOR',
    'HOLIDAY DECOR - EASTER': 'DECOR',
    'HOME KITCHEN': 'HOME',
    'KIDS': 'KIDS ACCESSORIES',
    'KIDS APPAREL': 'KIDS CLOTHING',
    'KIDS HAIR': 'KIDS ACCESSORIES',
    'KIDS HAT': 'KIDS ACCESSORIES',
    'KIDS HATS': 'KIDS ACCESSORIES',
    'KIDS JEWELRY': 'KIDS ACCESSORIES'
}

new_sales_data['Category'] = new_sales_data['Category'].replace(category_updates)

# Drop columns 'Type' and 'Theme' as they will not be used for prediction
data_cleaned = new_sales_data.drop(columns=['Type', 'Theme'])

data_cleaned

#Train Data


## first pass on random forest model

In [ ]:
# Vectorize the 'Description' column using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features for simplicity
X_tfidf = tfidf_vectorizer.fit_transform(data_cleaned['Description'])

# Create a DataFrame from the TF-IDF features
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Add other numerical features to the TF-IDF DataFrame
X = pd.concat([X_tfidf_df, data_cleaned[['Retail Price', 'Sold Price']].reset_index(drop=True)], axis=1)

# Encode the target variable 'Category'
y = data_cleaned['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model's performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

accuracy_rf, report_rf

##NLP and tuning on Description field as input to prediction variable

In [ ]:
from sklearn.model_selection import GridSearchCV

# Vectorize the 'Description' column using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features for simplicity
X_tfidf = tfidf_vectorizer.fit_transform(data_cleaned['Description'])

# Create a DataFrame from the TF-IDF features
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Add other numerical features to the TF-IDF DataFrame
X = pd.concat([X_tfidf_df, data_cleaned[['Retail Price', 'Sold Price']].reset_index(drop=True)], axis=1)

# Encode the target variable 'Category'
y = data_cleaned['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

# Initialize the GridSearchCV with Random Forest model
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the best model with best parameters
best_model = RandomForestClassifier(random_state=42, **best_params)
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_best = best_model.predict(X_test)

# Evaluate the model's performance
best_accuracy = accuracy_score(y_test, y_pred_best)
best_report = classification_report(y_test, y_pred_best)

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)
print("Test Set Accuracy:", best_accuracy)
print("Classification Report:\n", best_report)

## best parameters for hyper tuning


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

# Apply preprocessing to the 'Description' column
data_cleaned['Description'] = data_cleaned['Description'].apply(preprocess_text)

# Define a TF-IDF vectorizer with optimized parameters
tfidf_vectorizer = TfidfVectorizer(max_features=500, max_df=0.95, min_df=5, ngram_range=(1, 2))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_cleaned['Description'], data_cleaned['Category'], test_size=0.2, random_state=42)

# Define a pipeline with TF-IDF vectorizer and Random Forest classifier
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', RandomForestClassifier(random_state=42))
])

# Define a parameter grid for GridSearchCV
param_grid = {
    'tfidf__max_df': [0.85, 0.9, 0.95],
    'tfidf__min_df': [3, 5, 7],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20],
    'clf__min_samples_split': [2, 5]
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Display the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Add the predictions and prediction probabilities to the original DataFrame
data_cleaned['Predicted_Category'] = grid_search.predict(data_cleaned['Description'])
pred_probabilities = grid_search.predict_proba(data_cleaned['Description'])
max_probabilities = pred_probabilities.max(axis=1)
data_cleaned['Category_prediction_probability'] = max_probabilities

# Display the first 20 rows of the output
print(data_cleaned[['Description', 'Predicted_Category', 'Category_prediction_probability']].head(20))


##run with Best NLP parameters

In [ ]:
# Vectorize the 'Description' column using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 features for simplicity
X_tfidf = tfidf_vectorizer.fit_transform(data_cleaned['Description'])

# Create a DataFrame from the TF-IDF features
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Add other numerical features to the TF-IDF DataFrame
X = pd.concat([X_tfidf_df, data_cleaned[['Retail Price', 'Sold Price']].reset_index(drop=True)], axis=1)

# Encode the target variable 'Category'
y = data_cleaned['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the best parameters found by GridSearchCV
best_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

# Initialize the Random Forest model with the best parameters
best_model = RandomForestClassifier(random_state=42, **best_params)

# Train the model
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_best = best_model.predict(X_test)

# Evaluate the model's performance
best_accuracy = accuracy_score(y_test, y_pred_best)
best_report = classification_report(y_test, y_pred_best)

print("Best Parameters:", best_params)
print("Test Set Accuracy:", best_accuracy)
print("Classification Report:\n", best_report)

##best feature chart

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract feature importances from the best model
feature_importances = best_model.feature_importances_

# Create a DataFrame for the feature importances
feature_names = X.columns
feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(12, 8))
sns.set(style='darkgrid')
plt.style.use('dark_background')
ax = sns.barplot(x='Importance', y='Feature', data=feature_importances_df.head(20), palette='Blues_r')
plt.title('Top 20 Feature Importances in Random Forest Model', color='black')
plt.xlabel('Importance', color='black')
plt.ylabel('Feature', color='black')

# Customize the appearance of the plot
ax.tick_params(colors='black')  # Set color of the tick labels
ax.yaxis.label.set_color('black')  # Set the y-axis label color
ax.xaxis.label.set_color('black')  # Set the x-axis label color
plt.setp(ax.get_xticklabels(), color='black')  # Set x-tick labels color
plt.setp(ax.get_yticklabels(), color='black')  # Set y-tick labels color
ax.spines['top'].set_color('black')
ax.spines['right'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')

plt.show()


## writing model to local drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import joblib

# Assuming tfidf_vectorizer is the vectorizer used during model training
joblib.dump(tfidf_vectorizer, '/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')

##load the file - for use later

In [ ]:
import joblib

# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')


## run with optimized paramters - writing best model to outputs

In [9]:
# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)


# Apply preprocessing to the 'Description' column
data_cleaned['Description'] = data_cleaned['Description'].apply(preprocess_text)

# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')

# Define the Random Forest classifier with the optimized parameters
rf_classifier = RandomForestClassifier(max_depth=None, min_samples_split=2, n_estimators=100, random_state=42)

# Define the pipeline with the optimized parameters
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', rf_classifier)
])

# Fit the pipeline with the training data (if you have a training set)
# Split the data into training and testing sets (update this if you have training data available)
X_train, X_test, y_train, y_test = train_test_split(data_cleaned['Description'], data_cleaned['Category'], test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Save the pipeline to Google Drive
joblib.dump(pipeline,'/content/drive/My Drive/Bexa/best_model_category.pkl')


# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_category.pkl')

# Transform the 'Description' column using the fitted TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(data_cleaned['Description'])

# Make predictions on the new data
data_cleaned['Predicted_Category'] = pipeline.predict(data_cleaned['Description'])

# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(data_cleaned['Description'])

# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)

# Add the probabilities to the DataFrame
data_cleaned['Category_prediction_probability'] = max_probabilities

# Display the first 20 rows of the output with the new predicted_category and probability columns
print(data_cleaned[['Description', 'Predicted_Category', 'Category_prediction_probability']].head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Mounted at /content/drive


NameError: name 'data_cleaned' is not defined

#Run the model


In [10]:
#import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
import joblib
from google.colab import drive

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)


# Filter out rows where 'Retail Price' has a value of 'Retail Price'
model_output = union_data[union_data['Retail Price'] != 'Retail Price']

model_output = model_output.drop(columns=['Type', 'Theme', 'Category'])

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

model_output['Retail Price'] = model_output['Retail Price'].apply(clean_price)
model_output['Sold Price'] = model_output['Sold Price'].apply(clean_price)
model_output['Store Amount'] = model_output['Store Amount'].apply(clean_price)
model_output['Consignor Amount'] = model_output['Consignor Amount'].apply(clean_price)

# Apply preprocessing to the 'Description' column
model_output['Description'] = model_output['Description'].apply(preprocess_text)

# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')


# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_category.pkl')


# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])

# Make predictions on the new data
model_output['Predicted_Category'] = pipeline.predict(model_output['Description'])

# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])

# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)

# Add the probabilities to the DataFrame
model_output['Category_prediction_probability'] = max_probabilities


# Remove all '#' from the 'Consignor Amount' column, replace empty strings with '0', and convert to float
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('[\$,#]', '', regex=True)
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('', '0').astype(float)

# Handle date conversion
#model_output['Date'] = pd.to_datetime(union_data['Date'], infer_datetime_format=True, errors='coerce')


# Function to ensure four-digit year
def ensure_four_digit_year(date_str):
    # Match two-digit year and convert to four-digit year
    date_str = re.sub(r'(\d{1,2}/\d{1,2}/)(\d{2})$', r'\g<1>20\2', date_str)
    return date_str

# Apply the function to the 'Date' column
model_output['Date'] = model_output['Date'].apply(ensure_four_digit_year)

# Convert the adjusted date strings to datetime objects
model_output['Date'] = pd.to_datetime(model_output['Date'], format='%m/%d/%Y', errors='coerce')

# Display the first 20 rows of the combined data
print(model_output.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         Date                                        Description     SKU  \
0  2024-06-23  pink yellow champagne sequin beaded large tote...  NV7VAG   
1  2024-06-22                                  pin dog lover pin  2TESGR   
2  2024-06-22           bracelet red white blue stretch bracelet  THYGCK   
3  2024-06-22                         pink tiger acrylic earring  1HGLFS   
4  2024-06-22                                 earring small stud  1P8ABR   
5  2024-06-22                          huhhle hoop golod earring  R67TY3   
6  2024-06-22  kid hat kid mini era embroidered baseball cap ...  SXS8C9   
7  2024-06-22           earring married car acrylic pink earring  PCQP7B   
8  2024-06-22  earring golf iron ball cart set enamel post ea...  YNSJEP   
9  2024-06-22                earring set pickleball stud earring  DJT957   
10 2024-06-22            backpack r

#write to file on google sheets

In [11]:

# Assuming 'y_train' is a Series, convert it to a DataFrame for easier handling
model_output = model_output.astype(str)

# Convert the DataFrame to a list of lists
data = [model_output.columns.values.tolist()] + model_output.values.tolist()

# Create a new Google Sheet or open an existing one
spreadsheet_name = 'painted_tree_ml_training'
worksheet_name = 'Painted Tree - New model output June'

# Try to open the existing spreadsheet
try:
    spreadsheet = gc.open(spreadsheet_name)
except gspread.SpreadsheetNotFound:
    spreadsheet = gc.create(spreadsheet_name)

# Check if the worksheet already exists and delete it if it does
try:
    worksheet = spreadsheet.worksheet(worksheet_name)
    spreadsheet.del_worksheet(worksheet)
except gspread.WorksheetNotFound:
    pass

# Add a new worksheet
worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=len(model_output)+1, cols=len(model_output.columns))
#worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=len(y_train_df)+1, cols=len(y_train_df.columns))

# Write data to the worksheet
worksheet.update('A1', data)


<ipython-input-11-013281710f64>:29: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data)


{'spreadsheetId': '1gzZ4ffnM5Uc4cCHmAigMQ4IS3xxVokBc1wM5tQjPQcM',
 'updatedRange': "'Painted Tree - New model output June'!A1:K4822",
 'updatedRows': 4822,
 'updatedColumns': 11,
 'updatedCells': 53042}

In [ ]:
import matplotlib.pyplot as plt

# Plot the confidence distribution
plt.figure(figsize=(10, 6))
plt.hist(model_output['Category_prediction_probability'], bins=20, color='blue', edgecolor='black')
plt.title('Confidence Distribution')
plt.xlabel('Category_prediction_probability')
plt.ylabel('Frequency')
plt.grid(False)  # Eliminate gridlines
plt.show()


#data viz

In [ ]:


# Assume `monthly_sales` is your original DataFrame with a DateTime index
# Recreate `monthly_sales_grouped` with both year and month information
monthly_sales = model_output
monthly_sales_grouped = monthly_sales.resample('M').sum()
monthly_sales_grouped.index = pd.to_datetime(monthly_sales_grouped.index)
monthly_sales_grouped = monthly_sales_grouped.to_frame(name='Total Sales').reset_index()

# Extract year and month for plotting
monthly_sales_grouped['Year'] = monthly_sales_grouped['Date'].dt.year
monthly_sales_grouped['Month'] = monthly_sales_grouped['Date'].dt.strftime('%B')

# Create a trend chart with the updated title
plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_sales_grouped, x='Month', y='Total Sales', hue='Year', marker='o', palette='tab10')

# Set the title and labels
plt.title('Bexa Boutique Monthly Sales Trend 2024', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)

# Format the y-axis to display as dollars
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

# Remove grid lines and set white background
sns.set_style("white")
plt.gca().set_facecolor('white')
plt.grid(False)

# Show the plot
plt.show()


In [ ]:

# Assuming filtered_data_no_adjustments has a DateTime index
# Group sales by week
weekly_sales = filtered_data_no_adjustments['Sold Price'].resample('W').sum()

# Reset the index to have the Date as a column
weekly_sales = weekly_sales.reset_index()

# Extract year and week for plotting
weekly_sales['Year'] = weekly_sales['Date'].dt.year
weekly_sales['Week'] = weekly_sales['Date'].dt.strftime('%Y-%U')

# Create a trend chart with weekly sales
plt.figure(figsize=(16, 9))
sns.lineplot(data=weekly_sales, x='Date', y='Sold Price', hue='Year', marker='o', palette='tab10')

# Set the title and labels
plt.title('Bexa Boutique Weekly Sales Trend', fontsize=14)
plt.xlabel('Week', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)

# Format the y-axis to display as dollars
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

# Remove grid lines and set white background
sns.set_style("white")
plt.gca().set_facecolor('white')
plt.grid(False)

# Show the plot
plt.show()



In [ ]:

# Assuming filtered_data_no_adjustments has a DateTime index
# Group sales by week
weekly_sales = filtered_data_no_adjustments['Sold Price'].resample('W').sum()

# Reset the index to have the Date as a column
weekly_sales = weekly_sales.reset_index()

# Extract year and week number for plotting
weekly_sales['Year'] = weekly_sales['Date'].dt.year
weekly_sales['Week'] = weekly_sales['Date'].dt.strftime('Week %U')

# Normalize the weeks within the same calendar year
weekly_sales['Week'] = weekly_sales['Date'].dt.strftime('Week %U')

# Create a trend chart with weekly sales
plt.figure(figsize=(16, 9))
sns.lineplot(data=weekly_sales, x='Week', y='Sold Price', hue='Year', marker='o', palette='tab10')

# Set the title and labels
plt.title('Bexa Boutique Weekly Sales Trend', fontsize=14)
plt.xlabel('Week', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)

# Format the y-axis to display as dollars
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Remove grid lines and set white background
sns.set_style("white")
plt.gca().set_facecolor('white')
plt.grid(False)

# Show the plot
plt.show()


In [ ]:
# Define bins for consignor amounts
bins = [0, 10, 25, 75, float('inf')]
labels = ['$0 to $10', '$10 to $25', '$25 to $75', 'Above $75']

# Bin the consignor amounts and count the number of items in each bin
consignor_bins = pd.cut(filtered_data_no_adjustments['Consignor Amount'], bins=bins, labels=labels)
consignor_bin_counts = consignor_bins.value_counts().sort_index()

consignor_bin_counts

In [ ]:
# Create a histogram showing the distribution of consignor amounts in descending order
plt.figure(figsize=(10, 6))
sns.barplot(x=consignor_bin_counts.values, y=consignor_bin_counts.index, palette="Blues_r", order=consignor_bin_counts.index)

# Set the title and labels
plt.title('Distribution of Consignor Amounts', fontsize=16)
plt.xlabel('Number of Items', fontsize=14)
plt.ylabel('Consignor Amount Range', fontsize=14)

# Set dark background and remove grid lines
sns.set(style='darkgrid')
plt.style.use('dark_background')

# Show the plot
plt.show()


In [ ]:
# Calculate the percentage of items in each bin
consignor_bin_percentages = (consignor_bin_counts / consignor_bin_counts.sum()) * 100

# Create a pie chart showing the distribution of consignor amounts
plt.figure(figsize=(10, 6))
plt.pie(consignor_bin_percentages, labels=consignor_bin_percentages.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Blues_r"))

# Set the title
plt.title('Distribution of Consignor Amounts')

# Set white background
plt.gca().set_facecolor('white')
sns.set_style("white")
# Show the plot
plt.show()

consignor_bin_percentages


In [ ]:
# Calculate the percentage distribution of consignor amounts for each month
monthly_distribution = filtered_data_no_adjustments.groupby([filtered_data_no_adjustments.index.month, consignor_bins]).size().unstack().fillna(0)
monthly_percentages = monthly_distribution.div(monthly_distribution.sum(axis=1), axis=0) * 100

# Create a stacked bar chart
plt.figure(figsize=(14, 8))
monthly_percentages.plot(kind='bar', stacked=True, colormap='Blues_r', ax=plt.gca())

# Set the title and labels
plt.title('Monthly Distribution of Consignor Amounts', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Percentage of Items', fontsize=14)

# Set white background and remove grid lines
sns.set(style='white')
plt.gca().set_facecolor('white')
plt.grid(False)

# Set legend
plt.legend(title='Consignor Amount Range', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()


In [ ]:


# Define bins for consignor amounts
bins = [0, 10, 25, 75, float('inf')]
labels = ['$0 to $10', '$10 to $25', '$25 to $75', 'Above $75']

# Convert 'Consignor Amount' to numeric if not already
filtered_data_no_adjustments['Consignor Amount'] = pd.to_numeric(filtered_data_no_adjustments['Consignor Amount'], errors='coerce')

# Bin the consignor amounts and group by year and month
filtered_data_no_adjustments['Year'] = filtered_data_no_adjustments.index.year
filtered_data_no_adjustments['Month'] = filtered_data_no_adjustments.index.month
filtered_data_no_adjustments['Consignor Bin'] = pd.cut(filtered_data_no_adjustments['Consignor Amount'], bins=bins, labels=labels)

# Calculate the percentage distribution of consignor amounts for each month and year
monthly_distribution = filtered_data_no_adjustments.groupby(['Year', 'Month', 'Consignor Bin']).size().unstack().fillna(0)
monthly_percentages = monthly_distribution.div(monthly_distribution.sum(axis=1), axis=0) * 100

# Create a stacked bar chart for each year
years = monthly_percentages.index.get_level_values('Year').unique()

fig, axes = plt.subplots(len(years), 1, figsize=(14, 8 * len(years)), sharex=True)
if len(years) == 1:
    axes = [axes]

for ax, year in zip(axes, years):
    data = monthly_percentages.loc[year]
    data.plot(kind='bar', stacked=True, colormap='Blues_r', ax=ax)
    ax.set_title(f'Monthly Distribution of Consignor Amounts - {year}', fontsize=16)
    ax.set_xlabel('Month', fontsize=14)
    ax.set_ylabel('Percentage of Items', fontsize=14)
    ax.legend(title='Consignor Amount Range', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(False)
    ax.set_facecolor('white')

# Set common style and layout
sns.set(style='white')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Calculate the total consignor amount by item category
category_consignor_amount = filtered_data_no_adjustments.groupby('Category')['Consignor Amount'].sum()

# Remove grid lines and set white background
#plt.gca().set_facecolor('white')


# Identify the top five categories
top_categories = category_consignor_amount.nlargest(5)

# Group the remaining categories into 'Other'
other_categories_amount = category_consignor_amount[~category_consignor_amount.index.isin(top_categories.index)].sum()
grouped_data = pd.concat([top_categories, pd.Series(other_categories_amount, index=['Other'])])

# Create labels with the total consignor amount in dollars
labels = [f'{cat}: ${amt:,.2f}' for cat, amt in grouped_data.items()]

# Create a pie chart with the top five categories and 'Other'
plt.figure(figsize=(10, 6))
plt.pie(grouped_data, labels=labels, autopct='%1.1f%%', startangle=140)

# Set the title
plt.title('BEXA Boutique Total Sales by Item Category')

# Set white background
plt.gca().set_facecolor('white')

# Show the plot
plt.show()


In [ ]:
# Identify the top eight themes
# Calculate the total consignor amount by item theme
theme_consignor_amount = filtered_data_no_adjustments.groupby('Theme')['Consignor Amount'].sum()

top_themes = theme_consignor_amount.nlargest(8)

# Group the remaining themes into 'Other'
other_themes_amount = theme_consignor_amount[~theme_consignor_amount.index.isin(top_themes.index)].sum()
grouped_data_by_theme = pd.concat([top_themes, pd.Series(other_themes_amount, index=['Other'])])

# Create labels with the total consignor amount in dollars
labels_by_theme = [f'{theme}: ${amt:,.2f}' for theme, amt in grouped_data_by_theme.items()]

# Create a pie chart with the top eight themes and 'Other'
plt.figure(figsize=(10, 6))
plt.pie(grouped_data_by_theme, labels=labels_by_theme, autopct='%1.1f%%', startangle=140)

# Set the title
plt.title('BEXA Boutique Total Sales by Item Theme')

# Set white background
plt.gca().set_facecolor('white')

# Show the plot
plt.show()


In [ ]:
# Calculate the total consignor amount by item type
type_consignor_amount = filtered_data_no_adjustments.groupby('Type')['Consignor Amount'].sum()

# Identify the top eight types
top_types = type_consignor_amount.nlargest(8)

# Group the remaining types into 'Other'
other_types_amount = type_consignor_amount[~type_consignor_amount.index.isin(top_types.index)].sum()
grouped_data_by_type = pd.concat([top_types, pd.Series(other_types_amount, index=['Other'])])

# Create labels with the total consignor amount in dollars
labels_by_type = [f'{type}: ${amt:,.2f}' for type, amt in grouped_data_by_type.items()]

# Create a pie chart with the top eight types and 'Other'
plt.figure(figsize=(10, 6))
plt.pie(grouped_data_by_type, labels=labels_by_type, autopct='%1.1f%%', startangle=140)

# Set the title
plt.title('BEXA Boutique Total Sales by Item Type')

# Set white background
plt.gca().set_facecolor('white')

# Show the plot
plt.show()
# Calculate the total number of units sold by item
units_sold = filtered_data_no_adjustments.groupby('Description').size().sort_values(ascending=False)




In [ ]:
# Identify the top 10 items sold by number of units
top_10_items_sold = units_sold.head(10)

# Calculate the total consignor amount for the top 10 items sold by number of units
top_10_items_descriptions = top_10_items_sold.index
consignor_amounts_top_10 = filtered_data_no_adjustments[filtered_data_no_adjustments['Description'].isin(top_10_items_descriptions)]
total_consignor_amounts = consignor_amounts_top_10.groupby('Description')['Consignor Amount'].sum()

#top_10_items_sold

# Create a histogram of the top 10 items sold by number of units with consignor amounts and white labels
plt.figure(figsize=(12, 8))
sns.barplot(x=top_10_items_sold.values, y=top_10_items_sold.index, palette="Blues_r")

# Annotate the bars with the total consignor amounts
for i, (value, consignor_amount) in enumerate(zip(top_10_items_sold.values, total_consignor_amounts.values)):
    plt.text(value, i, f' ${consignor_amount:,.2f}', va='center', ha='left', fontsize=12, color='black')

# Set the title and labels
plt.title('Bexa Boutique Top 10 Items Sold by Number of Units', fontsize=16)
plt.xlabel('Number of Units Sold', fontsize=14)
plt.ylabel('Item Description', fontsize=14)

# Set white background and remove grid lines
sns.set(style='white')
plt.gca().set_facecolor('white')
plt.grid(False)
plt.style.use('default')

# Show the plot
plt.show()


In [ ]:
import numpy as np

# Create a basket representation of the data
basket = filtered_data_no_adjustments.groupby(['Invoice', 'Description'])['Sold Price'].sum().unstack().reset_index().fillna(0).set_index('Invoice')

# Convert to binary representation
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

# Calculate co-occurrence matrix
co_occurrence_matrix = basket_sets.T.dot(basket_sets)

# Remove diagonal elements to avoid self-association
co_occurrence_matrix = co_occurrence_matrix.mask(np.eye(co_occurrence_matrix.shape[0], dtype=bool))

# Display the top co-occurrences
top_co_occurrences = co_occurrence_matrix.stack().sort_values(ascending=False).head(10)
top_co_occurrences


In [ ]:
# Count the number of items per invoice
invoice_item_counts = filtered_data_no_adjustments.groupby('Invoice').size()

# Count the number of invoices with only one item and those with multiple items
single_item_invoices = invoice_item_counts[invoice_item_counts == 1].count()
multiple_item_invoices = invoice_item_counts[invoice_item_counts > 1].count()

#, multiple_item_invoices

# Create a pie chart for the number of single-item and multiple-item invoices
invoice_counts = pd.Series([single_item_invoices, multiple_item_invoices], index=['Single Item', 'Multiple Items'])

# Calculate the percentages
invoice_percentages = (invoice_counts / invoice_counts.sum()) * 100

# Create labels with the number of invoices and percentage
labels = [f'{label}: {count} ({percent:.1f}%)' for label, count, percent in zip(invoice_counts.index, invoice_counts.values, invoice_percentages.values)]

# Create the pie chart
plt.figure(figsize=(8, 6))
plt.pie(invoice_counts, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Blues_r"))

# Set the title
plt.title('Distribution of Invoices by Number of Items')

# Set white background
plt.gca().set_facecolor('white')

# Show the plot
plt.show()



In [ ]:
# Calculate the number of single-item and multiple-item invoices per month
monthly_invoice_counts = filtered_data_no_adjustments.groupby([filtered_data_no_adjustments.index.to_period('M'), 'Invoice']).size().unstack().fillna(0)
monthly_invoice_counts['Type'] = monthly_invoice_counts.sum(axis=1).apply(lambda x: 'Single Item' if x == 1 else 'Multiple Items')
monthly_distribution = monthly_invoice_counts.groupby([monthly_invoice_counts.index.to_timestamp(), 'Type']).size().unstack().fillna(0)

# Calculate the percentages
monthly_percentages = monthly_distribution.div(monthly_distribution.sum(axis=1), axis=0) * 100

# Create a stacked bar chart for the number of invoices and the percentage of total
fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot the number of invoices
monthly_distribution.plot(kind='bar', stacked=True, ax=ax1, color=sns.color_palette("Blues_r"))
ax1.set_ylabel('Number of Invoices', fontsize=14)
ax1.set_xlabel('Month', fontsize=14)
ax1.set_title('Monthly Distribution of Invoices by Number of Items', fontsize=16)
ax1.legend(title='Invoice Type', bbox_to_anchor=(1.05, 1), loc='upper left')

# Set white background and remove grid lines
sns.set(style='white')
plt.gca().set_facecolor('white')
plt.grid(False)

# Plot the percentage of total invoices on a secondary y-axis
ax2 = ax1.twinx()
monthly_percentages.plot(kind='bar', stacked=True, ax=ax2, alpha=0)
ax2.set_ylabel('Percentage of Invoices', fontsize=14)
ax2.yaxis.set_major_formatter(ticker.PercentFormatter())

plt.show()


In [ ]:
#from google.colab import auth
#auth.authenticate_user()


#get the current data that is tagged with category, theme and type from export for training
#location = 'Kentucky'
#location = 'Cincinnati'

#gc  = gspread.authorize(creds)

# read data and put it in a dataframe, by site which is defined above
#if location == 'Kentucky':
  #gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/117PfZR2st0KhQ1L4t3GuNL7vsj2CYq6MDJhMm-cr6N4/edit?gid=427915297#gid=427915297') ##Kentucky
  #sheets = gsheets.worksheet('Consignor Access - Activity (10)').get_all_values()
  #sales_data = pd.DataFrame(sheets[1:], columns=sheets[0])
#elif location == 'Cincinnati':
  #gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1r_xXGRpcoegezaQTwpoOXhgaWpbHZIJYDoVnygquSGQ') ##Cincinnati
  #sheets = gsheets.worksheet('Consignor Access - Activity (11)').get_all_values()
  #sales_data = pd.DataFrame(sheets[1:], columns=sheets[0])

#sales_data
# Display the first few rows and get the column names and data types
#sales_data_info = sales_data.info()
#sales_data_head = sales_data.head()

